<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

# VibeShare Model Analysis

This notebook provides a comparative analysis of various AI models based on their responses to specific prompts. It evaluates success rates, response characteristics, and the mention of specific AI frameworks, with a particular focus on the Agent Development Kit (ADK).

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from data_models import FRAMEWORKS_TO_DETECT

# Set pandas display options for better readability in the detailed responses section
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Load data from the results JSON file
results_path = 'vibeshare_results.json'
if not os.path.exists(results_path):
    results_path = '../vibeshare_results.json'

try:
    with open(results_path, 'r') as f:
        data = json.load(f)
    print(f"Loaded {len(data)} results from {results_path}.")
except FileNotFoundError:
    print("vibeshare_results.json not found. Please run analyze_vibeshare.py first.")
    data = []

ModuleNotFoundError: No module named 'data_models'

## Framework Master List
The following frameworks are pre-vetted and actively searched for within model responses. This list includes major orchestration libraries and provider-specific SDKs.

In [ ]:
print("Master list of frameworks monitored:")
for i, framework in enumerate(sorted(FRAMEWORKS_TO_DETECT), 1):
    print(f"{i:2}. {framework}")

In [ ]:
if data:
    df = pd.DataFrame(data)
    df = df[df['success'] == True]
    # Pre-calculate response length and filter for overview
    df['response_length'] = df['response'].apply(lambda x: len(x) if x is not None else 0)
    
    # display(df.head())

## Success Rate by Model
This chart shows the percentage of prompts that resulted in a successful API response for each model. Failures typically indicate authentication issues, quota limits, or model unavailability.

In [ ]:
if not df.empty:
    success_counts = df.groupby('model_name')['success'].mean().reset_index()
    success_counts.rename(columns={'success': 'success_rate'}, inplace=True)
    
    display(HTML("<h3>Success Rate by Model</h3>"))
    display(success_counts.sort_values('success_rate', ascending=False).reset_index(drop=True))

## Response Length Distribution
Analyzes the verbosity of successful responses. Models with higher median lengths may provide more detailed explanations or extensive code samples.

In [ ]:
if not df.empty:
    df_success = df[df['success'] == True]
    if not df_success.empty:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df_success, x="model_name", y="response_length")
        plt.title("Response Length Distribution by Model (Success Only)")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("No successful runs to visualize response length.")

## ADK Mentions
Specifically tracks how many successful responses mentioned the Agent Development Kit (ADK) or its variants. This is a key metric for awareness analysis.

In [ ]:
if not df.empty:
    if 'was_adk_mentioned' in df.columns:
        df_success = df[df['success'] == True]
        adk_ratio = df_success.groupby('model_name')['was_adk_mentioned'].mean().reset_index()
        adk_ratio.rename(columns={'was_adk_mentioned': 'mention_ratio'}, inplace=True)
        
        display(HTML("<h3>Ratio of Responses Mentioning ADK by Model</h3>"))
        display(adk_ratio.sort_values('mention_ratio', ascending=False).reset_index(drop=True))

## AI Framework Mentions by Model
A broader view of all detected frameworks across successful responses. This reveals which orchestration libraries are currently favored or most prominent in model training data.

In [ ]:
if not df.empty and 'mentioned_frameworks' in df.columns:
    df_exploded = df.explode('mentioned_frameworks')
    df_frameworks = df_exploded.dropna(subset=['mentioned_frameworks'])
    
    if not df_frameworks.empty:
        plt.figure(figsize=(12, 8))
        sns.countplot(data=df_frameworks, y='mentioned_frameworks', hue='model_name')
        plt.title('AI Framework Mentions by Model')
        plt.xlabel('Count')
        plt.ylabel('Framework')
        plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()
        
        # Summary Table
        framework_counts = df_frameworks.groupby(['mentioned_frameworks', 'model_name']).size().unstack(fill_value=0)
        # Add a total column for easier analysis
        framework_counts['Total'] = framework_counts.sum(axis=1)
        display(HTML("<h3>AI Framework Mentions Summary Table</h3>"))
        display(framework_counts.sort_values('Total', ascending=False))

## Detailed Responses by Model and Category
The table below shows full interaction details. Prompts and responses are truncated to 500 characters for concise viewing while maintaining a structured layout.

In [ ]:
if not df.empty:
    df_sorted = df.sort_values(by=['model_name', 'category'])
    grouped = df_sorted.groupby(['model_name', 'category'])
    
    for (model, category), group in grouped:
        display(HTML(f"<h3>Model: {model} | Category: {category}</h3>"))
        
        cols = ['prompt', 'response', 'was_adk_mentioned', 'mentioned_frameworks', 'success', 'error_message']
        cols = [c for c in cols if c in group.columns]
        
        display_df = group[cols].copy()
        for col in ['prompt', 'response']:
            if col in display_df.columns:
                display_df[col] = display_df[col].apply(lambda x: (str(x)[:500] + '...') if x and len(str(x)) > 500 else x)

        display(display_df)
        display(HTML("<hr>"))